1. IMPORT AND INSTALL DEPENDENCIES

In [ ]:
%pip install tensorflow==2.13.0 tensorflow-gpu opencv-python sklearn matplotlib mediapipe

In [2]:
import cv2
import numpy as np
import mediapipe as mp
import os
import time
from matplotlib import pyplot as plt

2. MARKING KEYPOINTS USING MEDIAPIPE HOLISTIC

In [3]:
mp_holistic = mp.solutions.holistic     #Mediapipe Holistic Model
mp_drawing = mp.solutions.drawing_utils #Mediapipe Drawing Utilities

In [88]:
def mediapipe_detection(image, model) :
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)      #Convert Image from BGR(Blue-Green-Red) to RGB(Red-Green-Blue)
  image.flags.writeable = False                       #Image is no longer writeable
  result = model.process(image)                       #Deploying mediapipe model to process landmarks
  image.flags.writeable = True                        #Changing image status to writeable
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)      #Converting Image from RGB back to BGR
  return image, result                                #Return result

In [64]:
def draw_landmarks(image, result) :
    mp_drawing.draw_landmarks(image, result.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                              mp_drawing.DrawingSpec(color = (80, 110, 10), thickness = 1, circle_radius = 1), 
                              mp_drawing.DrawingSpec(color = (80, 256, 121), thickness = 1, circle_radius = 1))  #Drawing Face Connections on Window
    mp_drawing.draw_landmarks(image, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  #Drawing Pose Connections on Window
    mp_drawing.draw_landmarks(image, result.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color = (121, 22, 76), thickness = 2, circle_radius = 4), 
                              mp_drawing.DrawingSpec(color = (121, 44, 250), thickness = 2, circle_radius = 2))  #Drawing Left Hand Connections on Window 
    mp_drawing.draw_landmarks(image, result.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color = (245, 117, 66), thickness = 2, circle_radius = 4), 
                              mp_drawing.DrawingSpec(color = (245, 66, 230), thickness = 2, circle_radius = 2))  #Drawing Right Hand Connections on Window

In [ ]:
#Open Camera
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: #Instance of Mediapipe Holistic Model 
    while cap.isOpened():
        #Read frame
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1) #Optional for fixing camera issues
        #Detecting Keypoints using Mediapipe detection function
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        #Rendering Detected Landmarks on Current Frame
        draw_landmarks(image, results)
        #Show on screen
        cv2.imshow('OpenCV Window', image)
        #Close Window
        if cv2.waitKey(10) & 0xFF== ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

3. EXTRACTING KEYPOINTS FROM LAST CAPTURED FRAME

In [90]:
def extract_keypoints(results) :
#Extracting Keypoints of Face Landmarks
    if results.face_landmarks :
        face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten()
    else :
        face = np.zeros(1404)
    #Extracting Keypoints of Pose Landmarks
    if results.pose_landmarks :
        pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
    else :
        pose = np.zeros(132)
    #Extracting Keypoints of Right Hand Landmarks
    if results.right_hand_landmarks :
        right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    else :
        right_hand = np.zeros(63)
    #Extracting Keypoints of Left Hand Landmarks
    if results.left_hand_landmarks :
        left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    else :
        left_hand = np.zeros(63)
    return np.concatenate([face, pose, right_hand, left_hand])